# Hong Kongese Language Identifier
This notebook contains modifications to make it run with the Hong Kongese language identification dataset. The only difference is that we do not load the English vectors because they will be useless on Hong Kongese.
This notebook uses a dataset with 4 each of Hong Kongese and Standard Chinese articles.

# 5 - Multi-class Sentiment Analysis

In all of the previous notebooks we have performed sentiment analysis on a dataset with only two classes, positive or negative. When we have only two classes our output can be a single scalar, bound between 0 and 1, that indicates what class an example belongs to. When we have more than 2 examples, our output must be a $C$ dimensional vector, where $C$ is the number of classes.

In this notebook, we'll be performing classification on a dataset with 6 classes. Note that this dataset isn't actually a sentiment analysis dataset, it's a dataset of questions and the task is to classify what category the question belongs to. However, everything covered in this notebook applies to any dataset with examples that contain an input sequence belonging to one of $C$ classes.

Below, we setup the fields, and load the dataset. 

The first difference is that we do not need to set the `dtype` in the `LABEL` field. When doing a mutli-class problem, PyTorch expects the labels to be numericalized `LongTensor`s. 

The second different is that we use `TREC` instead of `IMDB` to load the `TREC` dataset. The `fine_grained` argument allows us to use the fine-grained labels (of which there are 50 classes) or not (in which case they'll be 6 classes). You can change this how you please.

In [1]:
import torch
from torchtext import data
from torchtext import datasets
import random

SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

DATASET="4"

Custom tokenizer to simply split at character level.

In [2]:
def tokenizer(text): # create a tokenizer function
    return list(map(str, text.replace(" ", "")))

Load dataset from data/language directory.

In [3]:
TEXT = data.Field(tokenize=tokenizer)
LABEL = data.LabelField()

fields = {'language': ('label', LABEL), 'text': ('text', TEXT)}
train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path = 'data/language/' + DATASET,
                                        train = 'train.json',
                                        validation = 'valid.json',
                                        test = 'test.json',
                                        format = 'json',
                                        fields = fields
)

Let's look at one of the examples in the training set.

In [4]:
vars(train_data[-1])

{'label': 'zh',
 'text': ['請',
  '支',
  '持',
  '《',
  '眾',
  '新',
  '聞',
  '》',
  '，',
  '成',
  '為',
  '訂',
  '戶',
  '\xa0',
  '有',
  '一',
  '派',
  '史',
  '家',
  '認',
  '為',
  '文',
  '化',
  '大',
  '革',
  '命',
  '根',
  '本',
  '未',
  '結',
  '束',
  '過',
  '，',
  '共',
  '產',
  '黨',
  '的',
  '基',
  '因',
  '，',
  '思',
  '維',
  '方',
  '式',
  '沒',
  '變',
  '過',
  '。',
  '文',
  '革',
  '時',
  '，',
  '紅',
  '衞',
  '兵',
  '抄',
  '你',
  '家',
  '，',
  '找',
  '出',
  '半',
  '封',
  '幾',
  '十',
  '年',
  '前',
  '寄',
  '給',
  '國',
  '民',
  '黨',
  '的',
  '信',
  '，',
  '你',
  '就',
  '「',
  '冚',
  '扮',
  '爛',
  '全',
  '家',
  '剷',
  '除',
  '」',
  '！',
  '今',
  '天',
  '更',
  '厲',
  '害',
  '，',
  '電',
  '腦',
  '按',
  '一',
  '下',
  '，',
  '就',
  '問',
  '你',
  '，',
  '「',
  '你',
  '兩',
  '年',
  '前',
  '是',
  '不',
  '是',
  '在',
  '某',
  '某',
  '宣',
  '言',
  '上',
  '簽',
  '過',
  '字',
  '？',
  '你',
  '幾',
  '年',
  '前',
  '為',
  '什',
  '麼',
  '去',
  '台',
  '灣',
  '出',
  '席',
  '台',
  '獨',
  '份',
  '子',
  '論',
  '壇',


Next, we'll build the vocabulary. As this dataset is small (only ~3800 training examples) it also has a very small vocabulary (~7500 unique tokens), this means we do not need to set a `max_size` on the vocabulary as before.

In [5]:
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

Next, we can check the labels.

The 6 labels (for the non-fine-grained case) correspond to the 6 types of questions in the dataset:
- `HUM` for questions about humans
- `ENTY` for questions about entities
- `DESC` for questions asking you for a description 
- `NUM` for questions where the answer is numerical
- `LOC` for questions where the answer is a location
- `ABBR` for questions asking about abbreviations

In [6]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x1221ed268>, {'hky': 0, 'zh': 1, 'en': 2})


As always, we set up the iterators.

In [7]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, 
    device=device,
    sort_key=lambda x: len(x.text), # the BucketIterator needs to be told what function it should use to group the data.
    sort_within_batch=False)

We'll be using the CNN model from the previous notebook, however any of the models covered in these tutorials will work on this dataset. The only difference is now the `output_dim` will be $C$ instead of $1$.

In [8]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        text = text.permute(1, 0)
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim=1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

We define our model, making sure to set `OUTPUT_DIM` to $C$. We can get $C$ easily by using the size of the `LABEL` vocab, much like we used the length of the `TEXT` vocab to get the size of the vocabulary of the input.

In [9]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT)

Another different to the previous notebooks is our loss function (aka criterion). Before we used `BCEWithLogitsLoss`, however now we use `CrossEntropyLoss`. Without going into too much detail, `CrossEntropyLoss` performs a *softmax* function over our model outputs and the loss is given by the *cross entropy* between that and the label.

Generally:
- `CrossEntropyLoss` is used when our examples exclusively belong to one of $C$ classes
- `BCEWithLogitsLoss` is used when our examples exclusively belong to only 2 classes (0 and 1) and is also used in the case where our examples belong to between 0 and $C$ classes (aka multilabel classification).

In [10]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

Before, we had a function that calculated accuracy in the binary label case, where we said if the value was over 0.5 then we would assume it is positive. In the case where we have more than 2 classes, our model outputs a $C$ dimensional vector, where the value of each element is the beleief that the example belongs to that class. 

For example, in our labels we have: 'HUM' = 0, 'ENTY' = 1, 'DESC' = 2, 'NUM' = 3, 'LOC' = 4 and 'ABBR' = 5. If the output of our model was something like: **[5.1, 0.3, 0.1, 2.1, 0.2, 0.6]** this means that the model strongly believes the example belongs to class 0, a question about a human, and slightly believes the example belongs to class 3, a numerical question.

We calculate the accuracy by performing an `argmax` to get the index of the maximum value in the prediction for each element in the batch, and then counting how many times this equals the actual label. We then average this across the batch.

In [11]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim=1, keepdim=True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum()/torch.FloatTensor([y.shape[0]])

The training loop is similar to before, without the need to `squeeze` the model predictions as `CrossEntropyLoss` expects the input to be **[batch size, n classes]** and the label to be **[batch size]**.

The label needs to be a `LongTensor`, which it is by default as we did not set the `dtype` to a `FloatTensor` as before.

In [12]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text)
        
        loss = criterion(predictions, batch.label)
        
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

The evaluation loop is, again, similar to before.

In [13]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text)
            
            loss = criterion(predictions, batch.label)
            
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Next, we train our model.

In [14]:
%%time

N_EPOCHS = 30

lowest_valid_loss = 100
for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the model with the lowest validation loss for use later
    saved = False
    if valid_loss < lowest_valid_loss:
        lowest_valid_loss = valid_loss
        with open("./models/language-identifier-" + DATASET + "-best.pt", 'wb') as fb:
            saved = True
            torch.save(model, fb)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% | Saved: {saved}')
    
with open("./models/language-identifier-" + DATASET + "-final.pt", 'wb') as ff:
    torch.save(model, ff)    

/anaconda3/envs/keras/lib/python3.6/site-packages/torch/serialization.py:250: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


| Epoch: 01 | Train Loss: 1.200 | Train Acc: 55.56% | Val. Loss: 0.986 | Val. Acc: 41.61% | Saved: True
| Epoch: 02 | Train Loss: 1.053 | Train Acc: 66.67% | Val. Loss: 0.923 | Val. Acc: 42.79% | Saved: True
| Epoch: 03 | Train Loss: 1.050 | Train Acc: 44.44% | Val. Loss: 0.856 | Val. Acc: 54.82% | Saved: True
| Epoch: 04 | Train Loss: 0.524 | Train Acc: 66.67% | Val. Loss: 0.814 | Val. Acc: 65.29% | Saved: True
| Epoch: 05 | Train Loss: 0.562 | Train Acc: 88.89% | Val. Loss: 0.799 | Val. Acc: 47.99% | Saved: True
| Epoch: 06 | Train Loss: 0.450 | Train Acc: 88.89% | Val. Loss: 0.790 | Val. Acc: 47.60% | Saved: True
| Epoch: 07 | Train Loss: 0.393 | Train Acc: 88.89% | Val. Loss: 0.768 | Val. Acc: 47.60% | Saved: True
| Epoch: 08 | Train Loss: 0.134 | Train Acc: 100.00% | Val. Loss: 0.745 | Val. Acc: 50.94% | Saved: True
| Epoch: 09 | Train Loss: 0.165 | Train Acc: 100.00% | Val. Loss: 0.728 | Val. Acc: 52.21% | Saved: True
| Epoch: 10 | Train Loss: 0.084 | Train Acc: 100.00% | Val. Lo

For the non-fine-grained case, we should get an accuracy of around 90%. For the fine-grained case, we should get around 70%.

In [15]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% |')

| Test Loss: 0.575 | Test Acc: 78.61% |


Deep learning tend to overfit the training data if it ran for too many epochs. We'll compare with the best model we've found.

In [16]:
with open("./models/language-identifier-" + DATASET + "-best.pt", 'rb') as fbl:
    best_model = torch.load(fbl)

In [17]:
best_model_test_loss, best_model_test_acc = evaluate(best_model, test_iterator, criterion)

print(f'| Test Loss: {best_model_test_loss:.3f} | Test Acc: {best_model_test_acc*100:.2f}% |')

| Test Loss: 0.575 | Test Acc: 78.61% |


Choose the model with the best accuracy.

In [18]:
if test_loss > best_model_test_loss:
    print("Will use best_model.")
    selected_model = best_model
else:
    print("Will use final model.")
    selected_model = model

Will use final model.


Similar to how we made a function to predict sentiment for any given sentences, we can now make a function that will predict the class of question given.

The only difference here is that instead of using a sigmoid function to squash the input between 0 and 1, we use the `argmax` to get the highest predicted class index. We then use this index with the label vocab to get the human readable label.

In [19]:
def predict_sentiment(sentence, trained_model, min_len=4):
    tokenized = tokenizer(sentence)
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = trained_model(tensor)
    print(preds)
    max_preds = preds.argmax(dim=1)
    return max_preds.item()

Now, let's try it out on a few different questions...

In [20]:
pred_class = predict_sentiment("特朗普上周四（7日）曾表示，在3月1日達成貿易協議的最後期限前，他不會與中國國家主席習近平會晤。", selected_model)
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

tensor([[ 1.1346,  1.8136, -1.0352]], grad_fn=<AddmmBackward>)
Predicted class is: 1 = zh


In [21]:
pred_class = predict_sentiment("喺未有互聯網之前，你老母叫你做人唔好太高眼角，正正常常嘅男人嫁出去就算。", selected_model)
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

tensor([[ 1.7034,  1.4114, -0.7012]], grad_fn=<AddmmBackward>)
Predicted class is: 0 = hky


In [22]:
pred_class = predict_sentiment("I need to get some food.", selected_model)
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

tensor([[-5.9854e-04,  2.7496e-01,  1.3226e+00]], grad_fn=<AddmmBackward>)
Predicted class is: 2 = en


In [23]:
def range_predictions(prelist, trained_model, min_len=4):
    min_len = 4
    predict_hky = {}
    predict_zh = {}
    predict_en = {}
    for token in prelist:
        tokenized = tokenizer(token)
        tokenized_len = len(tokenized)
        if tokenized_len < min_len:
            tokenized += ['<pad>'] * (min_len - tokenized_len)
        indexed = [TEXT.vocab.stoi[t] for t in tokenized]
        tensor = torch.LongTensor(indexed).to(device)
        tensor = tensor.unsqueeze(1)
        preds = trained_model(tensor)
        max_preds = preds.argmax(dim=1)
        if LABEL.vocab.itos[max_preds.item()] == 'hky':
            predict_hky[token] = preds.data[0][max_preds.item()].item()
        elif LABEL.vocab.itos[max_preds.item()] == 'zh':
            predict_zh[token] = preds.data[0][max_preds.item()].item()
        else:
            predict_en[token] = preds.data[0][max_preds.item()].item()
    return predict_hky, predict_zh, predict_en

In [24]:
predict_hky, predict_zh, predict_en = range_predictions(TEXT.vocab.itos, selected_model, min_len=4)

In [25]:
sorted_by_value = sorted(predict_hky.items(), key=lambda kv: kv[1])
sorted_by_value.reverse()
for i in range(5):
    if i < len(sorted_by_value):
        print(sorted_by_value[i])
    else:
        break

('<pad>', 0.36247241497039795)
('<unk>', 0.30481651425361633)


In [26]:
sorted_by_value = sorted(predict_zh.items(), key=lambda kv: kv[1])
sorted_by_value.reverse()
for i in range(5):
    if i < len(sorted_by_value):
        print(sorted_by_value[i])
    else:
        break

('新', 0.8408305048942566)
('障', 0.791587769985199)
('，', 0.7875770926475525)
('掹', 0.7861973643302917)
('問', 0.772991955280304)


In [27]:
sorted_by_value = sorted(predict_en.items(), key=lambda kv: kv[1])
sorted_by_value.reverse()
for i in range(5):
    if i < len(sorted_by_value):
        print(sorted_by_value[i])
    else:
        break

Check what kind of articles are incorrect.

In [28]:
with torch.no_grad():
    for batch in test_iterator:
        predictions = selected_model(batch.text)
        max_preds = predictions.argmax(dim=1)
        wrong_preds = (max_preds.eq(batch.label) == 0).nonzero()
        for wrong_pred in wrong_preds:
            wrong_idx = wrong_pred.item()
            incorrect_prediction = max_preds[wrong_idx].item()
            correct_label = batch.label[wrong_idx].item()
            print("Predicted \"" + LABEL.vocab.itos[incorrect_prediction] + "\" but should be \"" + LABEL.vocab.itos[correct_label] + "\".")
            text_i = batch.text[:,wrong_idx].tolist()
            text_striped_idx = [x for x in text_i if x != TEXT.vocab.stoi['<pad>']]
            full_text = list(map(lambda x: TEXT.vocab.itos[x], text_striped_idx))
            print("Article: ")
            print("".join(full_text))
            print()

Predicted "zh" but should be "hky".
Article: 
<unk>俊華原來冇話過佢一直認同<unk><unk>英？

Predicted "zh" but should be "hky".
Article: 
「<unk>子山精神」關「唱」鍾樹根乜嘢事？

Predicted "zh" but should be "hky".
Article: 
<unk>智思點解話一開始就得罪咗林鄭月<unk>？

Predicted "zh" but should be "hky".
Article: 
公務員啲工會點解冇理由講到自己咁<unk>大？

Predicted "zh" but should be "hky".
Article: 
今次有人動員臺港兩地搞羅<unk><unk>等人，有啲乜嘢深層次<unk>謀？

Predicted "zh" but should be "hky".
Article: 
對唔住，香港只得一個。我<unk>對所有真香港人。 <unk>畫系列：以畫<unk><unk>。作者facebook

Predicted "zh" but should be "hky".
Article: 
今日我哋講咗……<unk><unk><unk><unk><unk>下斯文面具<unk><unk>海報<unk>毒<unk>劉<unk><unk>笑港獨<unk><unk><unk>變相倒林鄭米<unk><unk>驚六百萬人<unk>有大學請<unk>得<unk>做教授

Predicted "zh" but should be "hky".
Article: 
就算依加影相有幾方便，都永遠唔夠一張即影即有嘅感動<unk>日本<unk>士率先披露將會係<unk><unk>登場嘅即影即有totebag，其中一面會有15個相位，除咗即影即有之外，仲可以放其他<unk>片或者相，用家可以自己<unk>造一個獨一無二嘅totebag！ 

Predicted "zh" but should be "hky".
Article: 
於2009年被上水鄉事委員會主席<unk>志強與地產代理萬<unk>成，以大量<unk>頭<unk>壞上水河上鄉農地興建馬路，八<unk>農婦劉愛<unk>入<unk>索償<unk>訴，法院判<unk>志強與

Predicted "zh" but should be "hky".
Article: 
上網讀新聞之際，老友阿黃剛好打電話約飯，他以前是小文青，後來做咗公務員，仲係幾高級那種，但聽說仍然<unk>爾翻翻書本。我順道提醒他關於<unk>上<unk>樹被評不<unk>之事，問他有乜意見。「向西<unk>上<unk>樹？唔係不<unk>咗好<unk>嗎？一直都包<unk><unk>的。」阿黃反問。我笑道：「唔係香港的向西<unk>上<unk>樹，係日本的<unk>上<unk>樹！」「<unk>？邊本？《<unk>威的<unk>林》？《1Q84》？佢本本都有好多<unk>嘢，點解遲唔審早唔審，<unk>家先嚟審咁離奇？」我道：「係《刺殺<unk>士團長》呀！都出<unk>咗將近一年，可能<unk>家先有人投訴<unk>。」「咁又唔係唔<unk>。有投訴就要做嘢，只要依例辦事就行，有乜問題？如果有人投訴都唔做嘢，咁先唔<unk>，分分鐘搞到自己畀人投訴。香港乃法治社會，依法辦事係<unk>心價值，要<unk>上<unk>樹包<unk><unk>，正正<unk>顯咗我們的法治精神，抵<unk>！」阿黃道。「吓，咁都得？」阿黃續道：「有乜唔得？時勢唔同啦，今時今日係我哋公務員的黃金盛世，有法在手，乜都得。求其一個AO都可以話唔畀你參選就唔畀你參選、話唔畀你組黨就唔畀你組黨，easy到無倫。評審一本書，<unk><unk>到唔<unk><unk>，更加唔使麻<unk>AO出手。你update吓自己啦，<unk>伯！」我開始無語了。這些年來，無語的事情太多，藏有道具<unk><unk>要被判<unk>、阿<unk>執<unk><unk>執一<unk>要被罰款、運輸管理一團<unk>的<unk><unk>從未道<unk>更不必下台、律政司長<unk>中<unk>建兼公然避<unk>……要叫區區一本<unk>上<unk>樹小說包<unk>，確係小case到唔小case，已經<unk>得勞氣，亦再也無氣可勞。阿黃見我沉默幾秒，安慰道：「<unk>伯，凡事要往積極面看、朝光明面看。<unk>上<unk>樹本本都<unk><unk>，<unk>家只係要一本包<unk>，而其他無事，讀書人應該<unk>幸特區政府於嚴<unk>執法之<unk>亦唔係唔講人情，之不過，萬一有人投訴埋他的其他作品

Predicted "zh" but should be "en".
Article: 
（中譯<unk>本在下）ThecurrentRioOlympicshassadlybeenbrandedasoneoftheugliestinrecentmemoriesasfarastheOlympicspiritisconcerned,orthelackofit.We’vesofarwitnessedmorethanourfairshareoftheunsightlysideofthisglobalsportingevent.BeforetheGamesevenstartedtherewerealreadyrisinganti-RussiansentimentsduetodopingscandalsandthenChineseswimmerSunYangtoppedit,notbybeingcalleda“drugcheat”,buttheensuingretaliatorywarofwordshelaunchedagainsthisaccuserthereafter,whichwasutterlyshameful.Fortunately,theGames’mostcelebratedmedallistMichaelPhelpssavedtheGames’PRimagefromspirallingoutofcontrolwithhisgraciousacknowledgementofhisdefeatinthe100-metrebutterflytoyoungfirst-timeOlympianJosephSchooling,whobecameafanofPhelps’aftermeetinghimeightyearsago.WithsuchadramaticturnofeventsunfoldingattheRioGames,Icouldn’tpossiblystandonthesidelinesandleavetheGamessosoon.Therefore,IdecidedtodedicateanothercolumntotheRioGames.Afterall,ittakesplaceonlyonceeveryfouryears.Whynot?Firstoff,“s